In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
# import yfinance as yf
#  from pomegranate import HiddenMarkovModel, State, NormalDistribution, GeneralMixtureModel
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy import optimize

from src.HMM import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Testing with discrete samples (known result)

In [13]:
Gamma = np.array([[0.7,0.3],[0.4,0.6]])
pi = np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])
delta = np.array([0.6,0.4])
sequence = np.array([0,1,0,2])

In [14]:
l = log_likelihood_discrete(delta, Gamma, pi, sequence)

In [15]:
np.exp(l)

0.009629600000000004

In [16]:
params = baum_welch(sequence, Gamma, pi, delta)

In [19]:
params

{'a': array([[0., 1.],
        [1., 0.]]),
 'b': array([[0. , 0.5, 0.5],
        [1. , 0. , 0. ]])}

### Load Data

In [7]:
# Using yfinance
# msci_world = yf.Ticker('URTH')
# msci_data = msci_world.history(period='max')
# msci_data.drop_duplicates(inplace=True)

# Loading from csv
msci_data = pd.read_csv('MSCI World Index_11_22_21-12_31_93.csv')

In [8]:
log_returns = np.log(1 + msci_data.set_index('Date').sort_index().pct_change().dropna().query('Date >= "1997-01-01"').Close.values)

In [9]:
N_eff = 260

In [10]:
f = 1 - 1/N_eff

In [ ]:
f**260

In [ ]:
weights = f**np.arange(N_eff, 0, -1)

In [ ]:
kmeans = KMeans(n_clusters=2).fit(log_returns[:,np.newaxis])

## Cell below can be used to guess mean and std dev of 2 states

In [ ]:
pi = []
for i in range(kmeans.n_clusters):
    X = log_returns[np.where(kmeans.labels_ == i)[0]]
    pi.append(NDist(kmeans.cluster_centers_[i][0],X.std()))

In [ ]:
pi_params = np.array([[0.05/252,0.11/np.sqrt(252)],[-0.1/252,0.3/np.sqrt(252)]]) 

In [ ]:
pi = [NDist(p[0],p[1]) for p in pi_params]

In [ ]:
r = np.random.randn(2,1)*0.01 + (1/2)
Gamma = np.hstack([r,1-r])
delta = np.random.randn(2,1)*0.01 + (1/2)

In [ ]:
l = log_likelihood(delta, Gamma, pi, log_returns[:N_eff], weights=None)

In [ ]:
l

In [ ]:
theta_0 = [delta[0][0],Gamma[0,1],Gamma[1,0],pi_params[0][0],pi_params[0][1],pi_params[1][0],pi_params[1][1]]

In [ ]:
theta_0

In [ ]:
foo = lambda x: -log_likelihood_optim(x, log_returns)

In [ ]:
theta = optimize.minimize(foo, np.array(theta_0), method='Nelder-Mead',bounds=[(0,1),(0,1),(0,1),(-0.05,0.05),(0.002,0.05),(-0.1,0.1),(0.002,0.05)])

In [ ]:
theta.x

In [ ]:
delta, Gamma, pi = vec_to_params(theta.x)

In [ ]:
log_likelihood(delta, Gamma, pi, log_returns)

In [ ]:
theta = optimize.minimize(foo, np.array(theta_0), method='trust-constr',bounds=[(0,1),(0,1),(0,1),(-0.05,0.05),(0.002,0.05),(-0.1,0.1),(0.002,0.05)])

In [ ]:
theta.x[6]

In [ ]:
theta_0_prime = [0.7,0.99,0.95,0.001,np.sqrt(1/252)*0.11,-0.005,np.sqrt(1/252)*0.3]

In [ ]:
theta_0_prime

In [ ]:
foo_prime = lambda x: -log_likelihood_optim(x,log_returns[:N_eff], weights=None)

In [ ]:
theta_prime = optimize.minimize(foo_prime, np.array(theta_0),method='Nelder-Mead', bounds=[(0,1),(0,1),(0,1),(-0.05,0.05),(0.002,0.05),(-0.1,0.1),(0.002,0.05)])

In [ ]:
theta_prime.x[2]#*np.sqrt(252)

### Calculating forecasts according to HMM

In [ ]:
deltaHat, GammaHat, piHat = vec_to_params(theta_prime.x)
forecasted_mean, forecasted_var = get_hmm_forecasts(100, deltaHat, GammaHat, piHat, log_returns)

In [ ]:
plt.plot(forecasted_mean*252)

In [ ]:
l, score, information = score_and_information(deltaHat, GammaHat, piHat, log_returns[:N_eff], weights=None)

In [ ]:
information

In [ ]:
theta_hat = params_to_vec(deltaHat, GammaHat, piHat)

In [ ]:
theta_hat + (1/N_eff)*np.linalg.inv(information)@score

In [ ]:
theta_hat

In [ ]:
deltaHat, GammaHat, piHat = vec_to_params(theta_prime.x)

In [ ]:
for i in range(1000):
    theta_hat = params_to_vec(deltaHat, GammaHat, piHat)
    l, score, information = score_and_information(deltaHat, GammaHat, piHat, log_returns[:N_eff], weights=None)
    print(l)
    theta_hat = theta_hat + (1/N_eff)*np.linalg.inv(information)@score
    theta_hat[0] = min(max(theta_hat[0],0),1)
    theta_hat[1] = min(max(theta_hat[1],0),1)
    theta_hat[2] = min(max(theta_hat[2],0),1)    
    deltaHat, GammaHat, piHat = vec_to_params(theta_hat)

In [ ]:
theta_hat

In [ ]:
theta_hat

In [ ]:
from hmmlearn import hmm

In [ ]:
model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=100000)

In [ ]:
model = model.fit(log_returns[:, np.newaxis])

In [ ]:
model.startprob_

In [ ]:
model.transmat_

In [ ]:
model.means_

In [ ]:
model.covars_

In [ ]:
theta_hat[2]

In [ ]:
deltaHat

In [ ]:
piHat[1]

In [ ]:
GammaHat

In [ ]:
model.score(log_returns[:,np.newaxis])

In [ ]:
pi = [NDist(model.means_[0],np.sqrt(model.covars_[0])),NDist(model.means_[1],np.sqrt(model.covars_[1]))]

In [ ]:
log_likelihood(model.startprob_, model.transmat_, pi, log_returns)